In [1]:
import numpy as np
import torch
import facenet_pytorch
import cv2 as cv
import matplotlib.pyplot as plt

In [2]:
from detect_fastmtcnn import FastMTCNN
detector = FastMTCNN(resize=0.5, margin=14, factor=0.5, keep_all=True, device="cuda")


def extract_face(filename, required_size=(160, 160)):
    image = cv.imread(filename)
    results = detector(image)
    faces_array = []
    for x, y, w, h in results:
        x *= 2
        y *= 2
        w *= 2
        h *= 2
        face = image[y:h, x:w]
        face = cv.resize(face, required_size)
        faces_array.append(face)
    return np.asarray(faces_array)

In [3]:
import os
from keras_facenet import FaceNet

model = FaceNet()


def load_face(dir, required_size=(160, 160)):
    faces = []
    for filename in os.listdir(dir):
        path = dir + filename
        face = extract_face(path, required_size)
        faces.append(face)
    faces = np.asarray(faces)
    return faces


def load_dataset(dir, required_size=(160, 160)):
    faces = None
    labels = None
    for subdir in os.listdir(path):
        faces_list = []
        labels_list = []
        for filename in os.listdir(path + subdir + "/"):
            file_path = path + subdir + "/" + filename
            print(file_path)
            face = extract_face(file_path)
            faces_list.extend(face)
            labels_list.extend([subdir for _ in range(len(face))])
        if faces is None:
            faces = np.asarray(faces_list)
            labels = np.asarray(labels_list)
        else:
            faces = np.concatenate((faces, faces_list))
            labels = np.concatenate((labels, labels_list))
    return faces, labels


def get_embedding_face(face):
    face = face.astype("float32")
    mean, std = face.mean(), face.std()
    face = (face - mean) / std

    samples = np.expand_dims(face, axis=0)
    yhat = model.embeddings(samples)
    yhat = np.asarray(yhat)
    return yhat[0]

In [4]:
def calculate_average_embeddings(faces, labels):
    unique_labels = np.unique(labels)
    average_embeddings = []

    for label in unique_labels:
        label_faces = faces[labels == label]
        embeddings = []

        for face in label_faces:
            embedding = get_embedding_face(face)
            embeddings.append(embedding)

        embeddings = np.array(embeddings)
        average_embedding = np.mean(embeddings, axis=0)
        average_embeddings.append(average_embedding)

    average_embeddings = np.array(average_embeddings)
    unique_labels = np.array(unique_labels)

    return average_embeddings, unique_labels

In [5]:
path = "dataset/"
faces = None
labels = None
for subdir in os.listdir(path):
    faces_list = []
    labels_list = []
    for filename in os.listdir(path + subdir + "/"):
        file_path = path + subdir + "/" + filename
        print(file_path)
        face = extract_face(file_path)
        faces_list.extend(face)
        labels_list.extend([subdir for _ in range(len(face))])
    if faces is None:
        faces = np.asarray(faces_list)
        labels = np.asarray(labels_list)
    else:
        faces = np.concatenate((faces, faces_list))
        labels = np.concatenate((labels, labels_list))

dataset/barack_obama/barack_obama_1.jpg
0.9999804496765137
0.7541635036468506
dataset/barack_obama/barack_obama_2.jpg
0.9999768733978271
dataset/barack_obama/barack_obama_3.jpg
0.9999946355819702
dataset/barack_obama/barack_obama_4.jpg
0.99996018409729
dataset/joe_biden/joe_biden_1.jpg
0.9990785121917725
dataset/joe_biden/joe_biden_2.jpg
0.9990211725234985


In [6]:
average_embeded_face, unique_labels = calculate_average_embeddings(faces, labels)
# embeddings = []
# for face in faces:
#     embedding = get_embedding_face(face)
#     embeddings.append(embedding)

1/1 [==============================] - 0s 39ms/step


In [8]:
for i in range(len(faces)):
    embed = get_embedding_face(faces[i])
    print(f'{model.compute_distance(embed, average_embeded_face[0])} - {labels[i]}')

1/1 [==============================] - 0s 33ms/step
0 - barack_obama
1/1 [==============================] - 0s 34ms/step
0 - barack_obama
1/1 [==============================] - 0s 33ms/step
0 - barack_obama
1/1 [==============================] - 0s 31ms/step
0 - barack_obama
1/1 [==============================] - 0s 33ms/step
0 - joe_biden
1/1 [==============================] - 0s 33ms/step
0 - joe_biden
